# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [ ]:
import numpy as np
import os
import datetime
import os
import cv2
from cv2 import imread, resize

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip /content/gdrive/MyDrive/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

In [ ]:
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 15

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    total_images = 30
    x = total_images # SV: Added
    y = 128 # SV: Added
    z = 128 # SV: Added
    img_idx = np.random.permutation(total_images)[:batch_size]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = total_images//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the image
                    starty = y//4
                    startz = z//4
                    image = image[startz:z-startz,starty:y-starty]
                    #resize the image
                    #image = cv2.resize(image, (128,128), interpolation = cv2.INTER_CUBIC)

                    image = image / 255. #normalize the image
                    batch_data[folder,idx,:,:,0] = cv2.resize(image[:,:,0], (y,z)) #resize and feed in the image
                    batch_data[folder,idx,:,:,1] = cv2.resize(image[:,:,1], (y,z)) #resize and feed in the image
                    batch_data[folder,idx,:,:,2] = cv2.resize(image[:,:,2], (y,z)) #resize and feed in the image

                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
            # write the code for the remaining data points which are left after full batches
            if total_images % batch_size != 0:
              remaining_data = total_images % batch_size
              batch_data = np.zeros((remaining_data,x,y,z,3))
              batch_labels = np.zeros((remaining_data,5))
              for folder in range(remaining_data):
                  imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                  for idx,item in enumerate(img_idx):
                      image = imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                      image = image / 255. #normalize the image
                      batch_data[folder,idx,:,:,0] = cv2.resize(image[:,:,0], (y,z)) #resize and feed in the image
                      batch_data[folder,idx,:,:,1] = cv2.resize(image[:,:,1], (y,z)) #resize and feed in the image
                      batch_data[folder,idx,:,:,2] = cv2.resize(image[:,:,2], (y,z)) #resize and feed in the image

                  batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
              yield batch_data, batch_labels



Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 #10
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# def conv3dmodel(inputdim):
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape= (30,128,128,3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.25))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization(center=True, scale=True))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='softmax'))
 # return model


Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [ ]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 28, 126, 126, 32)  2624      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 14, 63, 63, 32)   0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 63, 63, 32)   128       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 14, 63, 63, 32)    0         
                                                                 
 conv3d_1 (Conv3D)           (None, 12, 61, 61, 64)    55360     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 6, 30, 30, 64)    0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [ ]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [ ]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
# model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                     callbacks=callbacks_list, validation_data=val_generator, 
#                     validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

In [ ]:
### Transfer Learning
from tensorflow.keras.applications.resnet50 import ResNet50
resnet = ResNet50(include_top = False, weights = 'imagenet',input_shape = (128,128,3))

94765736/94765736 [==============================] - 5s 0us/step


In [ ]:
for layer in resnet.layers[:-10]:
  layer.trainable = False

In [ ]:
# model2 = Sequential([resnet,Flatten(),Dense(64,activation = 'relu'),Dense(5,activation = 'softmax')])

In [ ]:
# model2.summary()

In [ ]:
from tensorflow.keras.layers import Conv2D
resnet = ResNet50(include_top = False,weights = 'imagenet',input_shape = (128,128,3))
cnn = Sequential([resnet])
cnn.add(Conv2D(64,(2,2),strides=(1,1)))
cnn.add(Conv2D(16,(3,3),strides=(1,1)))
cnn.add(Flatten())

In [ ]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 3, 3, 64)          524352    
                                                                 
 conv2d_1 (Conv2D)           (None, 1, 1, 16)          9232      
                                                                 
 flatten_1 (Flatten)         (None, 16)                0         
                                                                 
Total params: 24,121,296
Trainable params: 24,068,176
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
model2 = Sequential()
model2.add(TimeDistributed(cnn,input_shape=(30,128,128,3)))
model2.add(GRU(16,input_shape=(None,30,16),return_sequences = True))
model2.add(GRU(8))
model2.add(Dense(5,activation = 'softmax'))

In [ ]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 30, 16)           24121296  
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 30, 16)            1632      
                                                                 
 gru_1 (GRU)                 (None, 8)                 624       
                                                                 
 dense_3 (Dense)             (None, 5)                 45        
                                                                 
Total params: 24,123,597
Trainable params: 24,070,477
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001)
model2.compile(loss='categorical_crossentropy',optimizer = opt, metrics = ['categorical_accuracy']  )

In [ ]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

<ipython-input-48-f6de87c31a59>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,


Source path =  /content/Project_data/train ; batch size = 15
Epoch 1/30
45/45 [==============================] - ETA: 0s - loss: 1.6217 - categorical_accuracy: 0.1659Source path =  /content/Project_data/val ; batch size = 15

Epoch 1: saving model to model_init_2023-02-1417_40_51.761184/model-00001-1.62166-0.16593-1.61419-0.31429.h5
45/45 [==============================] - 128s 2s/step - loss: 1.6217 - categorical_accuracy: 0.1659 - val_loss: 1.6142 - val_categorical_accuracy: 0.3143 - lr: 0.0010
Epoch 2/30
45/45 [==============================] - ETA: 0s - loss: 1.6136 - categorical_accuracy: 0.1985
Epoch 2: saving model to model_init_2023-02-1417_40_51.761184/model-00002-1.61361-0.19852-1.67539-0.20952.h5
45/45 [==============================] - 97s 2s/step - loss: 1.6136 - categorical_accuracy: 0.1985 - val_loss: 1.6754 - val_categorical_accuracy: 0.2095 - lr: 0.0010
Epoch 3/30
45/45 [==============================] - ETA: 0s - loss: 1.6162 - categorical_accuracy: 0.1926
Epoch 3: sa

In [ ]:
opt2 = optimizers.Adam()
model2.compile(loss='categorical_crossentropy',optimizer = opt2, metrics = ['categorical_accuracy']  )

In [ ]:
model2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)